In [2]:
%matplotlib inline

import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import mxnet as mx
import pandas as pd

In [1]:
from sklearn import cross_validation
import xgboost as xgb


/usr/local/lib/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def get_extractor():
    model = mx.model.FeedForward.load('/home/yuens/Downloads/code/mxnet_inference/resnet/resnet-50',\
                                      0, ctx=mx.cpu(), numpy_batch_size=1)
    # model.symbol.get_internals: 
    # Get a new grouped symbol whose output contains internal outputs of this symbol.
    fea_symbol = model.symbol.get_internals()["flatten0_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.cpu(), symbol=fea_symbol, numpy_batch_size=64,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor

In [3]:
def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

In [4]:
# 得到当前path路径的病人的所有图像序列
def get_data_id(path):
    sample_image = get_3d_data(path)
    sample_image[sample_image == -2000] = 0
    # f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    # 这里不是resampling而是遍历每个图像做
    for i in range(0, sample_image.shape[0] - 3, 3):
        # 每个tmp是三幅2D图像
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            # 直方图均衡化
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)
        # 每个tmp是三幅2D图像把结果追加保存到batch列表
        # tmp是3D的
        # batch是4D的(第i捆三张打包，三张打包中的第j张，第j张的第k行，第j张的第m列)
        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        # if cnt < 20:
        #     plots[cnt // 5, cnt % 5].axis('off')
        #     plots[cnt // 5, cnt % 5].imshow(np.swapaxes(tmp, 0, 2))
        # cnt += 1

    # plt.show()
    # 整个batch再次转化为numpy
    batch = np.array(batch)
    return batch

In [6]:
def calc_features():
    # 加载网络模型和参数
    net = get_extractor()
    # 逐个遍历病人
    # 获取当前病人的所有CT图像序列
    # 根据图像序列给网络做预测
    # 打印特征维度并将特征保存为np形式
    patients_folder_path = glob.glob('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1/*')
    print("len(patients_folder_path):{0}".format(len(patients_folder_path)))
    for idx in xrange(len(patients_folder_path[:10])):
        print(idx, patients_folder_path[idx]) 
    
    valid_pathients_folder_path = filter(lambda p: ".npy" not in p, patients_folder_path)
    extracted_patients_npy_folder_path = filter(lambda p: ".npy" in p, patients_folder_path)
    extracted_patients_folder_path = map(lambda p: p.replace(".npy", ""), extracted_patients_npy_folder_path)
    unextracted_patients_folder_path = filter(lambda p: p not in extracted_patients_folder_path, valid_pathients_folder_path)
    print("len(valid_pathients_folder_path):{0}".format(len(valid_pathients_folder_path)))
    for idx in xrange(len(valid_pathients_folder_path[:10])):
        print(idx, valid_pathients_folder_path[idx]) 
    
    print("len(unextracted_patients_folder_path):{0}".format(len(unextracted_patients_folder_path)))
    for idx in xrange(len(unextracted_patients_folder_path[:10])):
        print(idx, unextracted_patients_folder_path[idx])
        
        
    print("len(extracted_patients_folder_path):{0}".format(len(extracted_patients_folder_path)))
    for idx in xrange(len(extracted_patients_folder_path[:10])):
        print(idx, extracted_patients_folder_path[idx])
    
    for idx in xrange(len(unextracted_patients_folder_path)):
        folder_path = unextracted_patients_folder_path[idx]
        batch = get_data_id(folder_path)
        feats = net.predict(batch)
        print(idx,feats.shape)
        np.save(folder_path, feats)

In [16]:
def train_xgboost():
    import time
    df = pd.read_csv('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1_labels.csv')
    print(df.head())
    
    x = np.array([np.mean(np.load('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/npy_files/%s.npy' % str(id)),\
                          axis=0) for id in df['id'].tolist()]\
                )
    
    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)#0.20

    clf = xgb.XGBRegressor(max_depth=10,
                           n_estimators=10500,
                           min_child_weight=9,
                           learning_rate=0.0005,
                           nthread=8,
                           subsample=0.80,#0.8
                           colsample_bytree=0.80,
                           seed=4242)

    clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)],\
            verbose=True,\
            eval_metric='logloss',\
            early_stopping_rounds=250)
    # eval_metric='logloss' or 'error' 
    
#     # 输出模型验证集上的分类性能、
#     train_predictions = clf.predict(val_x)
#     #train_predprob = clf.predict_proba(train_x)#[:,1] #1的概率
#     print("\nModel Report")
#     print("val set")
#     print train_predictions
#     #print("AUC Score (Train): %f" % metrics.roc_auc_score(val_y, train_predprob))
    
#     # 训练集性呢过
#     print("train set")
#     train_predictions = clf.predict(trn_x)
#     #train_predprob = clf.predict_proba(trn_x)#[:,1] #1的概率
#     print("train set in train")
#     #print("Accuracy : %.4g" % metrics.accuracy_score(trn_y, train_predictions))
#     #print("AUC Score (Train): %f" % metrics.roc_auc_score(trn_y, train_predprob))
    
    return clf

In [17]:
def make_submit():
    clf = train_xgboost()
    
    # 取出测试集数据做预测
    df = pd.read_csv('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1_sample_submission.csv')
    x = np.array([np.mean(np.load('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/npy_files/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
    pred = clf.predict(x)

    # 取出训练集预测
#     df = pd.read_csv('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1_labels.csv')
#     x = np.array([np.mean(np.load('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/npy_files/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
#     pred = clf.predict(x)

    # 将结果写入文件准备提交
    df['cancer'] = pred
    df.to_csv('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/[mxnet+xgb]submit_metrics_logloss_axis_0_earlystopping_250_lr_0.0005.csv', index=False)
    print(df.head())

In [18]:
if __name__ == '__main__':
    # 单个特征的提取时间大概是50s左右
    #calc_features()
    import pandas as pd
    import numpy as np
    make_submit()

                                 id  cancer
0  0015ceb851d7251b8f399e39779d1e7d       1
1  0030a160d58723ff36d73f41b170ec21       0
2  003f41c78e6acfa92430a057ac0b306e       0
3  006b96310a37b36cccb2ab48d10b49a3       1
4  008464bb8521d09a42985dd8add3d0d2       1
[0]	validation_0-logloss:0.693053
Will train until validation_0-logloss hasn't improved in 250 rounds.
[1]	validation_0-logloss:0.692931
[2]	validation_0-logloss:0.692825
[3]	validation_0-logloss:0.692723
[4]	validation_0-logloss:0.692578
[5]	validation_0-logloss:0.692447
[6]	validation_0-logloss:0.692326
[7]	validation_0-logloss:0.692178
[8]	validation_0-logloss:0.692065
[9]	validation_0-logloss:0.691945
[10]	validation_0-logloss:0.691818
[11]	validation_0-logloss:0.691694
[12]	validation_0-logloss:0.691602
[13]	validation_0-logloss:0.691482
[14]	validation_0-logloss:0.691363
[15]	validation_0-logloss:0.691257
[16]	validation_0-logloss:0.691136
[17]	validation_0-logloss:0.691021
[18]	validation_0-logloss:0.690867
[19]	validat

In [129]:
df = pd.read_csv('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1_sample_submission.csv')

#filter(lambda patient_npy: ".npy" in patient_npy, )
x = np.array([np.mean(np.load('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

In [ ]:
def data_overview():
    patients_folder_path = glob.glob('/media/yuens/WIN10-ENTERTENMENT/Kaggle/DSB2017/stage1/*')
    print("len(patients_folder_path):{0}".format(len(patients_folder_path)))
    for idx in xrange(len(patients_folder_path[:10])):
        print(idx, patients_folder_path[idx]) 
    
    valid_pathients_folder_path = filter(lambda p: ".npy" not in p, patients_folder_path)
    extracted_patients_npy_folder_path = filter(lambda p: ".npy" in p, patients_folder_path)
    extracted_patients_folder_path = map(lambda p: p.replace(".npy", ""), extracted_patients_npy_folder_path)
    unextracted_patients_folder_path = filter(lambda p: p not in extracted_patients_folder_path, valid_pathients_folder_path)
    print("len(valid_pathients_folder_path):{0}".format(len(valid_pathients_folder_path)))
    for idx in xrange(len(valid_pathients_folder_path[:10])):
        print(idx, valid_pathients_folder_path[idx]) 
    
    print("len(unextracted_patients_folder_path):{0}".format(len(unextracted_patients_folder_path)))
    for idx in xrange(len(unextracted_patients_folder_path[:10])):
        print(idx, unextracted_patients_folder_path[idx])
        
        
    print("len(extracted_patients_folder_path):{0}".format(len(extracted_patients_folder_path)))
    for idx in xrange(len(extracted_patients_folder_path[:10])):
        print(idx, extracted_patients_folder_path[idx])
    
    for idx in xrange(len(unextracted_patients_folder_path)):
        folder_path = unextracted_patients_folder_path[idx]
        batch = get_data_id(folder_path)
        feats = net.predict(batch)
        print(idx,feats.shape)
        np.save(folder_path, feats)